# for pytorch 초보자

In [ ]:
''' import '''
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

## Tensor

In [1]:
''' init tensor'''
# tensor = numpy + GPU

# random
rnd_nor = torch.randn(2,3) # normal distribustion (mean=0, var=1)
rnd_uni = torch.rand(2,3)  # uniform distribustion (0~1)

print(rnd_nor)
print(rnd_nor * 5)
print(rnd_nor + rnd_nor)

print(rnd_nor.shape)
print(rnd_nor.dtype)


# other
z = torch.zeros([2,4])
o = torch.ones([2,4])
e = torch.eye(4)

print(z)
print(o)
print(e)

NameError: name 'torch' is not defined

In [ ]:
''' list vs tensor '''
lst1 = [1,2]
lst2 = [3,4]

tor1 = torch.tensor([1,2])
tor2 = torch.tensor([3,4])

print(lst1 + lst2)      # concat
print(tor1 + tor2)      # element-wise sum

print(torch.dot(tor1, tor2))        # dot   (= tor1.dot(tor2))
print(torch.matmul(tor1, tor2))     # matmul
print(torch.matmul(tor1.view(1,-1), tor2)[0])   # view : reshape와 유사, but contiguous() 해야 동작

[1, 2, 3, 4]
tensor([4, 6])
tensor(11)
tensor(11)
tensor(11)


In [ ]:
''' indexing, slicing '''
rnd1 = torch.randn(4,3)

print(rnd1)
print(rnd1[2, :])   # row vec
print(rnd1[:, 1])   # col vec
print(rnd1[2, 1])

tensor([[ 0.0911,  0.2355,  1.3368],
        [-0.0104,  0.3437, -1.4669],
        [-1.7265, -0.5412, -0.9112],
        [ 1.0339,  0.6922, -0.7014]])
tensor([-1.7265, -0.5412, -0.9112])
tensor([ 0.2355,  0.3437, -0.5412,  0.6922])
tensor(-0.5412)


In [ ]:
''' matmul '''
mat1 = torch.tensor([[2,5,8], [5,3,1]])     # (2,3)
mat2 = torch.tensor([[3,2], [5,4], [7,8]])  # (3,2)

print(mat1)
print(mat2)
print(torch.matmul(mat1, mat2)) # (2,2)     # matmul = @
print(torch.matmul(mat2, mat1)) # (3,3)

tensor([[2, 5, 8],
        [5, 3, 1]])
tensor([[3, 2],
        [5, 4],
        [7, 8]])
tensor([[87, 88],
        [37, 30]])
tensor([[16, 21, 26],
        [30, 37, 44],
        [54, 59, 64]])


In [ ]:
''' function '''

# operator
print(torch.sum(rnd1))
print(torch.mean(rnd1))
print(torch.var(rnd1))
print(torch.std(rnd1))

print(torch.sum(rnd1).item())   # item 추출 : tensor -> scalar


# condition
print(torch.where(rnd1>0, torch.ones(rnd1.shape), torch.zeros(rnd1.shape)))

tensor(-1.6244)
tensor(-0.1354)
tensor(0.9132)
tensor(0.9556)
-1.6243630647659302
tensor([[1., 1., 1.],
        [0., 1., 0.],
        [0., 0., 0.],
        [1., 1., 0.]])


In [ ]:
''' numpy <-> tensor '''
t1 = torch.ones(3)
n1 = t1.numpy()             # tensor -> numpy          
t2 = torch.from_numpy(n1)   # numpy -> torch

print(type(t1))
print(type(n1))
print(type(t2))

<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [ ]:
''' using GPU '''

# check gpu
print(torch.cuda.is_available())            # cuda
print(torch.cuda.device_count())

print(torch.backends.mps.is_available())    # mps
print(torch.backends.mps.is_built())

# torch.tensor([1, 2, 3, 4]).to("mps")        # test

False
0
True
True


In [ ]:
# load gpu
x_mps = torch.tensor([1,1], device='mps')   # to mps
# x_cuda = torch.tensor([1,1]).cuda()       # to cuda
x_cpu = x_mps.cpu()                         # to cpu

print(x_mps)
print(x_cpu)

# 다른장치(gpu/cpu) 간 연산 불가
# print(x_mps * x_cpu) # RuntimeError: Expected all tensors to be on the same device, but found at least two devices, mps:0 and cpu!

tensor([1, 1], device='mps:0')
tensor([1, 1])


## Computational Graph

In [ ]:
''' computational graph (auto-grad) '''
in1 = torch.tensor([2.], requires_grad=True)
in2 = torch.tensor([1.], requires_grad=True)
n1 = in1 + in2
n2 = in1 + 1
n3 = n1 * n2

print(in1)
print(in2)
print(n1)
print(n2)
print(n3)

tensor([2.], requires_grad=True)
tensor([1.], requires_grad=True)
tensor([3.], grad_fn=<AddBackward0>)
tensor([3.], grad_fn=<AddBackward0>)
tensor([9.], grad_fn=<MulBackward0>)


In [ ]:
# info
print(in1.data)
print(in1.grad)
print(in1.grad_fn)

tensor([2.])
None
None


In [ ]:
# without retain_grad
n3.backward(retain_graph=True)  # 연결된 모든 (이전)노드의 grad 계산
                                # retain_graph=True : backward() 후에도 그래프 유지, backward() 여러번 반복 가능
print(in1.grad)
print(in2.grad)
print(n1.grad)  # None  UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead.
print(n2.grad)  # None
print(n3.grad)  # None

tensor([6.])
tensor([3.])
None
None
None


/var/folders/dd/p4jm7_y14s3bydf6pp07ny2r0000gn/T/ipykernel_7286/2966265506.py:6: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_e3pikzc5fh/croot/libtorch_1738337599132/work/build/aten/src/ATen/core/TensorBody.h:494.)
  print(n1.grad)  # None  UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access

In [ ]:
# with retain_grad
n1.retain_grad()
n2.retain_grad()
n3.retain_grad()

n3.backward(retain_graph=True)

print(in1.grad)
print(in2.grad)
print(n1.grad)  # ok
print(n2.grad)  # ok
print(n3.grad)  # ok

tensor([12.])
tensor([6.])
tensor([3.])
tensor([3.])
tensor([1.])


In [ ]:
in1.grad.zero_() # zero_() : (grad 초기화 + inplace) (안하면 중첩)
in2.grad.zero_()
n1.grad.zero_()
n2.grad.zero_()
n3.grad.zero_()

print(in1.grad)
print(in2.grad)
print(n1.grad)
print(n2.grad)
print(n3.grad)

tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])


In [ ]:
in1.requires_grad = False   # grad 추적 중지
# in1 = in1.detach()

print(in1.requires_grad)

False


## Neural Network
nn.Module의 모든 w,b는 'requires_grad = True' 자동 설정

```py
# NN format
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

    def forward(self, x):
        return x
```

In [ ]:
''' Neural Network '''
import torch.nn as nn
import torch.optim as optim

# NN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=100, out_features=100, bias=True)
        self.fc1_act = nn.ReLU()
        self.fc2 = nn.Linear(100, 10, bias=True)

    def forward(self, x):
        out = self.fc1(x)
        out = self.fc1_act(out)
        out = self.fc2(out)
        return out

In [ ]:
# net
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc1_act): ReLU()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


In [ ]:
# parameters()
params = list(net.parameters())

print(len(params))      # 4 = 2 layer (weight, bias)
print(params[0].size())

4
torch.Size([100, 100])


In [ ]:
# named_parameters() : name + parameters
for name, param in net.named_parameters():
    print(name)

fc1.weight
fc1.bias
fc2.weight
fc2.bias


In [ ]:
# weight, bias
print(net.fc1.weight)   # =     / net.fc1.weight.data : tensor만 추출
print(params[0])        # =

Parameter containing:
tensor([[-0.0320,  0.0784, -0.0139,  ...,  0.0205,  0.0015,  0.0168],
        [ 0.0435, -0.0492, -0.0321,  ..., -0.0722, -0.0288,  0.0734],
        [-0.0923, -0.0093, -0.0047,  ...,  0.0270, -0.0882,  0.0003],
        ...,
        [-0.0754,  0.0526,  0.0936,  ..., -0.0461, -0.0519,  0.0386],
        [ 0.0818,  0.0842, -0.0654,  ...,  0.0374,  0.0411, -0.0520],
        [-0.0357,  0.0919, -0.0352,  ...,  0.0856, -0.0666, -0.0162]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.0828,  0.0971, -0.0881,  ...,  0.0926,  0.0925,  0.0142],
        [ 0.0976, -0.0198, -0.0380,  ..., -0.0061,  0.0847,  0.0833],
        [-0.0458,  0.0956, -0.0406,  ...,  0.0401, -0.0226,  0.0736],
        ...,
        [ 0.0365,  0.0564, -0.0675,  ..., -0.0030,  0.0017,  0.0307],
        [ 0.0414, -0.0438,  0.0053,  ...,  0.0679,  0.0763,  0.0040],
        [-0.0880, -0.0620, -0.0475,  ..., -0.0748, -0.0935, -0.0475]],
       requires_grad=True)


In [ ]:
''' train loop '''
batch_size = 10 # batch dim 필수
criterion = nn.MSELoss()

input = torch.randn(batch_size, 100)
target = torch.randn(batch_size, 10)

tensor(0.8984, grad_fn=<MseLossBackward0>)
before: None
after: tensor([ 0.0052,  0.0011,  0.0005, -0.0038, -0.0032,  0.0068, -0.0066, -0.0063,
        -0.0029,  0.0103, -0.0039, -0.0087,  0.0124,  0.0046,  0.0028,  0.0096,
         0.0078, -0.0014, -0.0010,  0.0025,  0.0065,  0.0036, -0.0068,  0.0116,
        -0.0136,  0.0035, -0.0096, -0.0021, -0.0001,  0.0092, -0.0056, -0.0018,
         0.0164, -0.0109,  0.0013,  0.0085,  0.0038, -0.0019,  0.0067,  0.0029,
        -0.0012,  0.0059,  0.0027, -0.0106,  0.0003, -0.0024,  0.0064, -0.0059,
         0.0107, -0.0072,  0.0153,  0.0134, -0.0042,  0.0011,  0.0067, -0.0080,
        -0.0044, -0.0028,  0.0036,  0.0112, -0.0136, -0.0009,  0.0045, -0.0023,
         0.0043, -0.0127,  0.0024, -0.0057,  0.0066,  0.0028,  0.0090, -0.0102,
         0.0082,  0.0027,  0.0010,  0.0069,  0.0075, -0.0107,  0.0044,  0.0038,
         0.0060,  0.0050,  0.0100, -0.0132,  0.0179,  0.0031,  0.0072, -0.0032,
        -0.0071, -0.0034,  0.0077, -0.0022, -0.0097,  0.0

In [ ]:
''' without optimizer '''

# iter
#   forward
#   backward
#       net.zero_grad()
#       loss.backward()
#   step

# forward
output = net(input)
loss = criterion(output, target)
print(loss)

# backward
net.zero_grad()
print('before:', net.fc1.bias.grad) # None
loss.backward()
print('after:', net.fc1.bias.grad)  # grad가 생김

# step
learning_rate = 0.01
for param in net.parameters():
    print(param.data.sub_(param.grad.data * learning_rate)) # sub()=빼기, sub_()=빼기+inplace

tensor(0.8984, grad_fn=<MseLossBackward0>)
before: None
after: tensor([ 0.0052,  0.0011,  0.0005, -0.0038, -0.0032,  0.0068, -0.0066, -0.0063,
        -0.0029,  0.0103, -0.0039, -0.0087,  0.0124,  0.0046,  0.0028,  0.0096,
         0.0078, -0.0014, -0.0010,  0.0025,  0.0065,  0.0036, -0.0068,  0.0116,
        -0.0136,  0.0035, -0.0096, -0.0021, -0.0001,  0.0092, -0.0056, -0.0018,
         0.0164, -0.0109,  0.0013,  0.0085,  0.0038, -0.0019,  0.0067,  0.0029,
        -0.0012,  0.0059,  0.0027, -0.0106,  0.0003, -0.0024,  0.0064, -0.0059,
         0.0107, -0.0072,  0.0153,  0.0134, -0.0042,  0.0011,  0.0067, -0.0080,
        -0.0044, -0.0028,  0.0036,  0.0112, -0.0136, -0.0009,  0.0045, -0.0023,
         0.0043, -0.0127,  0.0024, -0.0057,  0.0066,  0.0028,  0.0090, -0.0102,
         0.0082,  0.0027,  0.0010,  0.0069,  0.0075, -0.0107,  0.0044,  0.0038,
         0.0060,  0.0050,  0.0100, -0.0132,  0.0179,  0.0031,  0.0072, -0.0032,
        -0.0071, -0.0034,  0.0077, -0.0022, -0.0097,  0.0

In [ ]:
''' with optimizer '''

num_epochs = 200
optimizer = optim.SGD(net.parameters(), lr=0.01)

for i in range(num_epochs):

    # forward
    output = net(input)
    loss = criterion(output, target)
    print(loss)

    # backward
    optimizer.zero_grad()   # = net.zero_grad()
    loss.backward()

    # step
    optimizer.step()

tensor(4.0942e-07, grad_fn=<MseLossBackward0>)
tensor(4.0579e-07, grad_fn=<MseLossBackward0>)
tensor(4.0221e-07, grad_fn=<MseLossBackward0>)
tensor(3.9864e-07, grad_fn=<MseLossBackward0>)
tensor(3.9514e-07, grad_fn=<MseLossBackward0>)
tensor(3.9164e-07, grad_fn=<MseLossBackward0>)
tensor(3.8820e-07, grad_fn=<MseLossBackward0>)
tensor(3.8478e-07, grad_fn=<MseLossBackward0>)
tensor(3.8138e-07, grad_fn=<MseLossBackward0>)
tensor(3.7801e-07, grad_fn=<MseLossBackward0>)
tensor(3.7466e-07, grad_fn=<MseLossBackward0>)
tensor(3.7138e-07, grad_fn=<MseLossBackward0>)
tensor(3.6811e-07, grad_fn=<MseLossBackward0>)
tensor(3.6485e-07, grad_fn=<MseLossBackward0>)
tensor(3.6163e-07, grad_fn=<MseLossBackward0>)
tensor(3.5847e-07, grad_fn=<MseLossBackward0>)
tensor(3.5528e-07, grad_fn=<MseLossBackward0>)
tensor(3.5215e-07, grad_fn=<MseLossBackward0>)
tensor(3.4905e-07, grad_fn=<MseLossBackward0>)
tensor(3.4596e-07, grad_fn=<MseLossBackward0>)
tensor(3.4291e-07, grad_fn=<MseLossBackward0>)
tensor(3.3989